Explore the structure of the example .mat data file and finally export the regressor and response as xarray.

In [26]:
from pathlib import Path
import xarray as xr
import scipy
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [27]:
data_path = Path('../data/2975_LickingLama_20250207_125807.mat')
data = scipy.io.loadmat(data_path)
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'SessionData'])

In [28]:
# Load Vc.mat file (MATLAB v7.3 format requires h5py)
import h5py

vc_path = Path('../data/Vc.mat')
print(f"Loading variables from {vc_path}")

with h5py.File(vc_path, 'r') as f:
    # Load the specific variables
    Vc = f['Vc'][:]
    trialOn = f['trialOn'][:]
    U = f['U'][:]
    ledInfo = f['ledInfo'][:]  # Load as numpy array
    frameInfo = f['frameInfo'][:]  # Load as numpy array
    
    print(f"Vc shape: {Vc.shape}")
    print(f"trialOn shape: {trialOn.shape}")
    print(f"U shape: {U.shape}")
    print(f"ledInfo shape: {ledInfo.shape}")
    print(f"frameInfo shape: {frameInfo.shape}")  # Should be (39341, 2)

Loading variables from ../data/Vc.mat


KeyboardInterrupt: 

In [ ]:
print("SessionData field names:")
print(data['SessionData'].dtype.names)

Get event data from all trials

In [ ]:
event_data = data['SessionData']["RawEvents"][0][0][0][0][0][0]

Get the time of stimulus onsets. The same value repeats multiple times so this is probably relative to trial onset.

In [ ]:
stim_onsets = np.zeros(len(event_data))
for i, trial in enumerate(event_data):
    stim_onsets[i] = trial[0][0][0]["PlayStimulus"][0][0][0][0]
stim_onsets = stim_onsets
stim_onsets

Get the trial onset times (dimensions are frames, time stamps and trial number).

In [ ]:
import numpy as np

trialOn_clean = trialOn[:, ~np.isnan(trialOn[0, :])]
trialOn_clean = trialOn_clean.T
trialOn_clean


Calculate the sampling rate

In [ ]:
fs = (np.diff(trialOn_clean[:,0]) / np.diff(trialOn_clean[:,1])).mean()
fs

Combine times within trial with trial onset times. In the MATLAB code this was done like this

```matlab
stimTimes(iTrials) = bhv.RawEvents.Trial{iTrials}.States.PlayStimulus(1);
stimTimes(iTrials) = floor(stimTimes(iTrials) * frameRate) + trialTimes(iTrials);
```


Get sampling frequency and start time from `frameInfo`.

In [ ]:
# Calculate sampling rate from frameInfo
# sRate = 1 / mean(diff(frameInfo(:,2))); %sampling rate in Hz

# frameInfo is (2, 39341) - transposed from MATLAB (39341, 2)
# In MATLAB: frameInfo(:,2) means all rows, column 2 (timestamps)
# In Python with transposed data: frameInfo[1, :] means row 1, all columns (timestamps)

vc_start_time = frameInfo[1,0]
vc_srate = 1 / np.mean(np.diff(frameInfo[1, :]))  # sampling rate in Hz
vc_start_time, vc_srate

In [ ]:
stim_onsets_abs = stim_onsets + trialOn_clean[:,1] - vc_start_time
stim_onsets_abs_samples = np.floor(stim_onsets_abs * vc_srate).astype(int)
stim_onsets_abs_samples

In [ ]:
data['SessionData'][0,0]['StimType'], # 2 is for audio
data['SessionData'][0,0]['Rewarded']

In [ ]:
X = np.zeros(len(Vc))
X[stim_onsets_abs_samples] = 1
plt.plot(X[:1000])

In [ ]:
max_lag = 50
n = len(X)
X_lagged = np.zeros((n, max_lag))
for lag in range(max_lag):
    X_lagged[lag:, lag] = X[:n-lag]

In [ ]:
plt.imshow(X_lagged[:1000, :], aspect="auto")

In [ ]:
y = Vc[:, 0]
y.shape

No intercept, no regularization

In [ ]:
b = np.linalg.solve(X_lagged.T @ X_lagged, X_lagged.T @ y)

In [ ]:

time = np.arange(len(X)) / fs
ds = xr.Dataset(
    {
        "regressor": ("time", X),
        "response": ("time", y),
    },
    coords={"time": time},
    attrs={"fs": fs},
)
ds.to_netcdf("../data/data.nc")